# MaskRCNN for Chest X-ray Abnormalities Detection

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2

## Analyze preprocessed data

 [Dataset(1024x1024)](https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-png-1024x1024) is utilized. In the train_meta.csv, we have the original width and height as follow.

In [2]:
PREPROCESSED_DATA_DIR = '/home/daitran/Desktop/research/kaggle/VinBigData/preprocessed_data/1024_wh/'
PNG_1024_DIR = os.path.join(PREPROCESSED_DATA_DIR, 'train')
PNG_1024_CSV = os.path.join(PREPROCESSED_DATA_DIR, 'train_meta.csv')

PNG_512_WH_CSV = '/home/daitran/Desktop/research/kaggle/VinBigData/train/512_jpg/sample_dfq.csv'

DICOM_CSV = '/home/daitran/Desktop/research/kaggle/VinBigData/train/train.csv'

In [3]:
IMAGE_SIZE =1024

In [4]:
dicom_df = pd.read_csv(DICOM_CSV)
dicom_df = dicom_df.query('class_id != 14')
dicom_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
5,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
...,...,...,...,...,...,...,...,...
67903,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0
67906,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0
67907,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0
67908,db169d0be36123bd55b866d6aa73983b,Other lesion,9,R8,6.0,670.0,272.0,1736.0


In [5]:
png_1024_df = pd.read_csv(PNG_1024_CSV)
png_1024_df

,image_id,dim0,dim1
0,4d390e07733ba06e5ff07412f09c0a92,3000,3000
1,289f69f6462af4933308c275d07060f0,3072,3072
2,68335ee73e67706aa59b8b55b54b11a4,2836,2336
3,7ecd6f67f649f26c05805c8359f9e528,2952,2744
4,2229148faa205e881cf0d932755c9e40,2880,2304
...,...,...,...
14995,b3510302f95f75a91e0fd49e04767f02,3072,3072
14996,dcb081bb5e1dac41000e96fc37c8c322,2651,2507
14997,c619a784636c085eb798f98a5ba1102d,2875,2641
14998,e7ea94b2fec95a7461b10a7f3eea2897,3072,3072


In [6]:
png_512_wh_csv = pd.read_csv(PNG_512_WH_CSV)
png_512_wh_csv

,image_id,CategoryId,x_min,y_min,x_max,y_max,w_org,h_org
0,9a5094b2563a1ef3ff50dc5c7ff71345,3,170,301,406,401,2336,2080
1,9a5094b2563a1ef3ff50dc5c7ff71345,10,440,378,461,436,2336,2080
2,9a5094b2563a1ef3ff50dc5c7ff71345,11,440,378,461,436,2336,2080
3,9a5094b2563a1ef3ff50dc5c7ff71345,3,170,301,407,394,2336,2080
4,9a5094b2563a1ef3ff50dc5c7ff71345,3,169,287,410,386,2336,2080
...,...,...,...,...,...,...,...,...
404545,4b56bc6d22b192f075f13231419dfcc8,7,450,285,477,334,2040,1968
404546,4b56bc6d22b192f075f13231419dfcc8,0,284,102,360,180,2040,1968
404547,4b56bc6d22b192f075f13231419dfcc8,0,270,108,350,184,2040,1968
404548,4b56bc6d22b192f075f13231419dfcc8,3,198,265,432,330,2040,1968


In [7]:
abnormal_image_id_list = png_512_wh_csv['image_id'].unique()

In [8]:
len(abnormal_image_id_list)

4394

### Check accuracy of preprocessed data

In [9]:
# true_list = []
# for image_id_check in tqdm(abnormal_image_id_list):
# #     print(image_id_check)
#     w_1024 = int(png_1024_df.loc[png_1024_df['image_id'] == image_id_check]['dim0'].to_string(index = False))
#     w_512 = png_512_wh_csv.loc[png_512_wh_csv['image_id'] == image_id_check].iloc[0]['w_org']
    
#     h_1024 = int(png_1024_df.loc[png_1024_df['image_id'] == image_id_check]['dim1'].to_string(index = False))
#     h_512 = png_512_wh_csv.loc[png_512_wh_csv['image_id'] == image_id_check].iloc[0]['h_org']
    
    
#     if w_1024 == w_512 and h_1024 == h_512:
#         true_list.append(1)
    

In [10]:
# len(true_list)

## Convert to MaskRCNN format

In [11]:
def get_mask(img_dimensions, x_min, y_min, x_max, y_max):
    img_height, img_width = img_dimensions
    img_mask = np.full((img_height,img_width),0)
    img_mask[y_min:y_max,x_min:x_max] = 255
    return img_mask.astype(np.float32)

def rle_encoding(x):
    dots = np.where(x.T.flatten() == 255)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join([str(x) for x in run_lengths])

In [12]:
diagnostic_per_image = []
with tqdm(total=len(dicom_df)) as pbar:
    for idx,row in dicom_df.iterrows():
        image_id = row.image_id
        
#         print(image_id)
        image_df = dicom_df.query("image_id==@image_id")
        class_list = []
        RLE_list = []
        
        w_1024 = int(png_1024_df.loc[png_1024_df['image_id'] == image_id]['dim0'].to_string(index = False))
        h_1024 = int(png_1024_df.loc[png_1024_df['image_id'] == image_id]['dim1'].to_string(index = False))
        
        for diagnostic_id, diagnostic in image_df.iterrows():
            class_list.append(diagnostic.class_id)
            
            image_dimensions = (w_1024, h_1024)
#             print(image_dimensions)

            mask = get_mask(image_dimensions, int(diagnostic.x_min), int(diagnostic.y_min), int(diagnostic.x_max), int(diagnostic.y_max))
            resized_mask = cv2.resize(mask, (IMAGE_SIZE,IMAGE_SIZE))
            RLE_list.append(rle_encoding(resized_mask))
        diagnostic_per_image.append({"image_id":image_id,
                                     "CategoryId":class_list,
                                     "EncodedPixels":RLE_list})
        pbar.update(1)


100%|██████████| 36096/36096 [4:34:51<00:00,  2.19it/s]  


In [13]:
samples_df = pd.DataFrame(diagnostic_per_image)
samples_df["Height"] = IMAGE_SIZE
samples_df["Width"] = IMAGE_SIZE
samples_df.head()

,image_id,CategoryId,EncodedPixels,Height,Width
0,9a5094b2563a1ef3ff50dc5c7ff71345,"[3, 10, 11, 3, 3, 0]",[348764 199 349788 199 350812 199 351836 199 3...,1024,1024
1,051132a778e61a86eb147c7c6f564dfe,"[0, 0, 3, 3, 11, 0, 3]",[575753 98 576777 98 577801 98 578825 98 57984...,1024,1024
2,1c32170b4af4ce1a3030eb8167753b06,"[11, 13, 11, 13, 13, 13, 11, 13, 13]",[259192 25 260216 25 261240 25 262264 25 26328...,1024,1024
3,0c7a38f293d5f5e4846aa4ca6db4daf1,"[5, 5, 8, 5, 9]",[618595 771 619619 771 620643 771 621667 771 6...,1024,1024
4,47ed17dcb2cbeec15182ed335a8b5a9e,"[8, 9, 8, 9, 9, 8, 7, 7]",[228047 40 229071 40 230095 40 231119 40 23214...,1024,1024


## Check correctness

In [15]:
os.getcwd()

'/home/daitran/Desktop/git/chest_x_ray_abnormalities_detection/MaskRCNN_implementation'

In [16]:
samples_df.to_csv('1024_encoded_maskrcnn.csv', index = False)